# Introdução ao Machine Learning

In [ ]:
import os # Os para manipular itens do sistema operacional
import numpy as np # NumPy para manipular matrizes e vetores
import pandas as pd # Pandas para manipular os dataframes

# Plotar figuras
import matplotlib as mpl 
import matplotlib.pyplot as plt
import seaborn as sns

print('Setup completo!')

### Etapas de construção de um modelo de ML:

- Formulação do problema
- Seleção e adaptação do banco de dados
- Pré-processamento do banco de dados
- Separação do banco de dados em dados de treinamento e validação
- Ajuste do modelo com os dados de treinamento
- Validação do modelo e estimativa de desempenho com os dados de validação
- Implementação do modelo para aplicação

#### 1ª Formulação do problema

Estimar o valor médio das casas em um distrito da Califórnia, dado um determinado conjunto de dados e características deste distrito.

Fonte: https://www.kaggle.com/camnugent/california-housing-prices

Variáveis do banco de dados:

- longitude: Longitude
- latitude - Latitude
- housing_median_age - Idade média das construções no quarteirão
- total_rooms - Número de cômodos no quarteirão
- total_bedrooms - Número de quartos no quarteirão
- population - Número de pessoas que vivem no quarteirão
- households - Número de famílias vivendo no mesmo quarteirão
- median_income - Renda média mensal do quarteirão em que a casa está situada (medida em dezenas de milhares de dólares)
- median_house_value (target) - Valor médio das casas do quarteirão em que a casa está situada (em dólares)
- ocean_proximity - Proximidade da casa do oceano/mar

#### 2ª Seleção e adaptação do banco de dados

In [ ]:
# baixar o dataset do GitHub
!wget https://github.com/nias-ufv/elt574-aprendizado-de-maquina/raw/main/semana-1/housing.csv

In [ ]:
df_housing = pd.read_csv('./dataset/housing.csv')
df_housing.head() # 5 primeiras linhas

In [ ]:
df_housing.info() # Tipo das varíaveis

In [ ]:
df_housing['ocean_proximity'].value_counts() # Agrupa pelo ocean_proximity

In [ ]:
df_housing.loc[lambda i: i['ocean_proximity'] == 'ISLAND'] # Filtra por ilhas

In [ ]:
df_housing.describe() # Medidas, tendência e dispersão

In [ ]:
df_housing.hist(bins=50, figsize=(20, 15))
plt.show() # Plota os dados de cada coluna

In [ ]:
sns.histplot(data=df_housing, x='median_house_value', kde=True) # Suavização da coluna median_house_value usando sns

In [ ]:
df_housing.plot(kind='scatter', x='longitude', y='latitude', alpha=0.1) # Gráfico com as posições geográficas das casas

In [ ]:
# Grafico com as posicoes geograficas das casas e marcadores com 40% da cor e modulados pelo 'population'/100 e coloridos de acordo com o valor das casas
df_housing.plot(
    kind='scatter', 
    x='longitude', 
    y='latitude', 
    alpha=0.4,
    s=df_housing['population']/100, 
    label='Population', 
    figsize=(15,10), 
    c='median_house_value',
    cmap=plt.get_cmap('jet'),
    colorbar=True
)
plt.legend()

In [ ]:
# (Extra) Dispersão das amostras com o mapa da California

import folium
from folium.plugins import MarkerCluster

df_aux = df_housing
map_cahousing = folium.Map(
    location=[
        df_aux['latitude'].median(), 
        df_aux['longitude'].median()
    ],
    zoom_start=6,
    zoom_control=False, 
    width=800, 
    height=500
)

marker_cluster = MarkerCluster().add_to(map_cahousing)

for _, location_info in df_aux.loc[0:,:].iterrows(): 
    folium.Marker(
        [
            location_info['latitude'], 
            location_info['longitude']
        ],
        popup=folium.Popup(
            f"House value: U${location_info['median_house_value']}",
            max_width=500
        ),
        icon=folium.Icon(
            prefix='fa',
            icon='fa-house'
        ),
        overlay=True
    ).add_to(marker_cluster)

map_cahousing

In [ ]:
corr_matrix = df_housing.corr(numeric_only=True)
corr_matrix # Matriz de correlação

In [ ]:
corr_matrix['median_house_value'].sort_values(ascending=False) # Correlação das entradas com a saída desejada

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ['median_house_value', 'median_income', 'total_rooms', 'housing_median_age']
scatter_matrix(df_housing[attributes],figsize=(20,15))

In [ ]:
df_housing.plot(kind='scatter', x='median_income', y='median_house_value', alpha=0.1, figsize=(20,15))

In [ ]:
# Transformando variaveis categoricas
df_housing_cat = df_housing[['ocean_proximity']] # Repassa o vetor com os indices
df_housing_cat.value_counts()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
df_housing_cat_encoded = ordinal_encoder.fit_transform(df_housing_cat)
df_housing_cat_encoded

In [ ]:
ordinal_encoder.categories_

In [ ]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
df_housing_cat_1hot = cat_encoder.fit_transform(df_housing_cat)
df_housing_cat_1hot

In [ ]:
df_housing_cat_1hot.toarray()

In [ ]:
df_housing.describe() # Identificando variaveis com dados ausentes ou invalidos

In [ ]:
# Tres formas de fazer o tratamento
df_housing_cleaning = df_housing 

# 1- Retira as linhas com dados invalidos
# df_housing_cleaning.dropna(subset=['total_bedrooms'])

# 2- Descarta a coluna 'total_bedrooms'
# df_housing_cleaning.drop('total_bedrooms', axis=1)

# 3- Imputa (insere) dados onde esta faltando
median = df_housing['total_bedrooms'].median() # calcula a media dos dados validos
df_housing_cleaning['total_bedrooms'].fillna(median, inplace=True) #substitui os valores invalidos pela media do grupo valido

In [ ]:
df_housing.columns

In [ ]:
# Criando novas variaveis para analisar o problema
df_housing["rooms_per_household"] = df_housing["total_rooms"] / df_housing["households"]
df_housing["population_per_household"]= df_housing["population"] / df_housing["households"]

corr_matrix = df_housing.corr(numeric_only=True)
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
df_housing.plot(kind='scatter', x='rooms_per_household',y='median_house_value', alpha = 0.1)
plt.axis([0,5,0,520000])
plt.show()